# Importing Libraries

In [1]:
import gurobi as gp
from gurobi import *
import numpy as np
import pandas as pd
from datetime import datetime

# Reading Data

In [2]:
distances_data = pd.read_csv('distance_mtx_final.csv', index_col=0)
distances_data.head()


,65009,65419,65411,65429,65371,65379,65249,65241,65389,65381,...,65359,65071,65079,65081,65089,65091,65099,65439,65431,65149
65009,0,2106,2507,2063,2744,2564,1170,1592,1573,1999,...,1971,825,1106,1852,866,2174,1238,1082,2027,1595
65419,2354,0,387,746,637,458,1621,1350,1230,952,...,1713,2796,2362,1805,1907,1437,1619,3068,3303,3226
65411,1902,388,0,359,1026,846,1233,941,843,1259,...,1964,2316,1881,1324,1519,1744,1231,2588,2822,2380
65429,1913,813,2063,0,1451,1271,874,1885,1228,2292,...,2265,1935,1479,1691,1160,2129,1532,2206,2421,2667
65371,2419,856,468,827,0,588,1701,1409,1311,1017,...,1777,2861,2427,1870,1987,1502,1699,3133,3368,3290


In [3]:
duration_data = pd.read_csv('duration_matrix_final.csv', index_col=0)
duration_data.head()

,65009,65419,65411,65429,65371,65379,65249,65241,65389,65381,...,65359,65071,65079,65081,65089,65091,65099,65439,65431,65149
65009,0.000000,6.283333,7.566667,5.950000,8.500000,7.883333,4.800000,5.40,5.433333,6.366667,...,6.950000,3.900000,4.900000,6.100000,3.533333,7.500000,4.600000,4.333333,4.950000,5.966667
65419,8.133333,0.000000,1.683333,3.016667,2.216667,1.600000,5.250000,4.70,3.616667,3.150000,...,5.016667,8.583333,7.266667,4.800000,5.783333,3.833333,4.700000,9.183333,7.416667,8.483333
65411,6.750000,1.450000,0.000000,1.333333,3.683333,3.066667,3.566667,2.75,1.950000,3.550000,...,5.216667,6.983333,5.666667,3.216667,4.100000,4.266667,2.866667,7.583333,5.816667,7.350000
65429,6.850000,2.950000,5.966667,0.000000,5.166667,4.550000,2.233333,4.65,3.816667,5.616667,...,6.183333,6.300000,4.666667,4.816667,2.766667,6.150000,3.850000,6.900000,4.916667,8.233333
65371,7.983333,3.100000,1.650000,2.983333,0.000000,2.033333,5.216667,4.40,3.583333,2.983333,...,4.850000,8.416667,7.100000,4.650000,5.750000,3.666667,4.516667,9.016667,7.250000,8.316667


In [4]:
dist_array = np.asarray(distances_data) # Filtering for the relevant 200 stops
print("Shape of the stop-distances matrix is:", dist_array.shape)

dur_array = np.asarray(duration_data) # Filtering for the relevant 200 stops
print("Shape of the stop-durations matrix is:", dur_array.shape)

Shape of the stop-distances matrix is: (70, 70)
Shape of the stop-durations matrix is: (70, 70)


# Adjusting Parameters

In [5]:
# Adjustable Parameters
n = len(dist_array) # Number of Stops
L = 40
K = 5
buses = 2
max_duration = 60
M = 10000

print(f"Number of Stops (n): {n}\nMaximum number of stops visited by a single bus (L): {L} \nMinimum number of stops a bus can visit (K): {K} \nNumber of buses we can operate : {buses} ")

Number of Stops (n): 70
Maximum number of stops visited by a single bus (L): 40 
Minimum number of stops a bus can visit (K): 5 
Number of buses we can operate : 2 


# Initializing Model

In [6]:
def model_setup(buses, max_duration):    
    model = gp.Model("Bus_Route_Optimization")

    ######################
    ## Defining Decision Variables
    ######################
    x = model.addVars(n, n, vtype = GRB.BINARY, name = [str(i)+">"+str(j) for i in range(n) for j in range(n)])
    u = model.addVars(n, vtype = GRB.INTEGER, name = "u")
    t = model.addVars(n, name = "t")
    ######################
    ## Cost Calculation
    ######################
    """
    # Fuel cost = 2.73 $ per liter diesel
    # Mileage = 6.5 km / litre - diesel bus
    # Hybrid diesel bus 25% more efficient - Mileage = 8.125 km / litre
    # Cost to travel 1 meter (cost_m) = 2.73/(8.125x1000) = 0.0004336 $/meter
    # Cost of procuring each diesel-hybrid bus (cost_bus) = 0.6 million dollars
    # Maintenance cost per meter (maintanance_m) = 0.000385 $/meter
    # For traversing the network for 6 months (with every route traversed on average of 15mins, operating 5am till 12am)
    # cost_travel = cost_travel_route * hours_operation_day * routes_hour * days
    """
    cost_m, maint_cost, hours_operation_day, bus_freq, days, bus_cost = 0.0004336, 0.000385, 19, 15, 180, 600000 
    #cost_travel = gp.quicksum(dist_array[i,j]*x[i,j]*cost_m for i in range(n) for j in range(n)) * hours_operation_day * (60/bus_freq) * (days)

    cost = gp.quicksum(dist_array[i,j]*x[i,j] for i in range(n) for j in range(n))
    #cost_buses = gp.quicksum(x[0,j] * bus_cost for j in range(n)) * round((max_duration/bus_freq),0)

    ######################
    ## Objective Function
    ######################
    model.setObjective(cost, GRB.MINIMIZE) # We want to minimize the total cost

    ######################
    ## Constraints
    ######################
    # All buses start from terminus stop [0]
    model.addConstr(gp.quicksum(x[0, j] for j in range(1, n)) == buses, name = 'Minimum buses start from terminus node')


    # All buses end at terminus stop [0]
    model.addConstr(gp.quicksum(x[j, 0] for j in range(1, n)) == buses, name = 'Minimum buses end at terminus node') 
    

    # Each stop exited by only one bus
    for i in range(1, n):
        model.addConstr(gp.quicksum(x[i,j] for j in range(n)) == 1, name = 'Each Stop Exited by only 1 bus')

    # Each stop entered by only one bus
    for j in range(1, n):
        model.addConstr(gp.quicksum(x[i,j] for i in range(n)) == 1, name = 'Each Stop Entered by only 1 bus')

    # To not get stuck at the same stop
    for i in range(n):
        model.addConstr(x[i,i]==0)

    # Sub-tour elimination: Lifted Kulkarni–Bhave SECs
    for i in range(1,n):
        for j in range(1,n):
            if i==j:
                continue
            else:
                model.addConstr(u[i]-u[j]+ L*x[i,j] + (L-2)*x[j,i] <= L-1, name = 'Sub-tour elimination')

    # Ensure that buses visit at most L stops and at least K stops 
    for i in range(1,n):
        model.addConstr(u[i] + (L-2) * x[0, i] - x[i, 0] <= L-1, name = 'Visit at most L stops')
        model.addConstr(u[i] + x[0, i] + (2-K) * x[i, 0] >= 2, name = 'Visit at least K stops')
        if K < 4:
            model.addConstr(x[0,i] + x[i,0] <= 1) # Constraint is redundant for K >= 4

    # avg duration per bus route
    for i in range(n):
        for j in range(1,n):
            if i==j:
                continue
            else:
                model.addConstr(t[i]-t[j] + dur_array[i,j]  <= M*(1 - x[i,j]), name = 'sub_tour elimination with time')
                #model.addConstr(t[j]-t[i] + dur_array[j,i]  <= M*(1 - x[j,i]), name = 'sub_tour elimination with time')
                #model.addConstr(t[i]-t[j] + M*x[i,j]+ (M - dur_array[j,i])*x[j,i]  <= M - x[i,j]*dur_array[i,j] , name = 'sub_tour elimination with time')
    model.addConstrs(( t[i] + dur_array[i,0] - max_duration <= M*(1 - x[i,0]) for i in range(1,n) ), name = 'max_time_route')
    model.addConstrs(( t[i] - dur_array[0,i] >= M*(x[0,i] - 1) for i in range(1,n) ), name = 'initialise t1')
    model.addConstrs(( t[i] - dur_array[0,i] <= M*(1 - x[0,i]) for i in range(1,n) ), name = 'initialise t2')
    model.update()
    return model

# Running the optimization
## For 2 buses and within 57 mins per route

In [7]:
######################
## Running the optimization
######################

# model.Params.MIPGap = 0.02 # Ranges from 0 to 1. At 0.01 or 1% gap we want to stop the solver.
# model.Params.MIPFocus = 3 # To get a faster solution
# model.Params.Cuts = 3 # Cutting Planes ensure a faster way to solve complex problems
# model.Params.Presolve = 2 # Presolving reduces the size of the problem by substitution and eliminating redundant constraints 

time_1 = datetime.now() # Start-time
model = model_setup(buses, max_duration)
model.optimize() # Run model optimization
time_2 = datetime.now() # End-time

Set parameter Username
Academic license - for non-commercial use only - expires 2023-10-16
Gurobi Optimizer version 9.5.2 build v9.5.2rc0 (mac64[arm])
Thread count: 10 physical cores, 10 logical processors, using up to 10 threads
Optimize a model with 10008 rows, 5040 columns and 43747 nonzeros
Model fingerprint: 0xf82c2b44
Variable types: 70 continuous, 4970 integer (4900 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+04]
  Objective range  [2e+02, 7e+03]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+04]
Presolve removed 139 rows and 72 columns
Presolve time: 0.04s
Presolved: 9869 rows, 4968 columns, 43332 nonzeros
Variable types: 69 continuous, 4899 integer (4830 binary)

Root relaxation: objective 3.366490e+04, 423 iterations, 0.01 seconds (0.03 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 33664.9048    0   49          -

# Results

In [8]:
# Calculating Time Difference
delta_t = time_2 - time_1
print("Time Elapsed (Hours:Minutes:Seconds): ", str(delta_t))

Time Elapsed (Hours:Minutes:Seconds):  0:00:04.821191


In [9]:
# Code to get the stops traversed and their lengths
def results(buses):
    starting_paths = []
    for var in model.getVars():
        if  var.varName.startswith('0>') and var.X > 0:
            starting_paths.append(var.varName)
    d_starting_paths = {}
    for i in starting_paths:
        d_starting_paths[i] = None
        print(d_starting_paths)
    def cycle(filter_list):
        cycle = ['65009', ' > ']
        k = 0
        for i in range(1,n):
            for j in range(1,n):
                name = str(k)+'>'+str(j)
                if name not in filter_list:
                    if model.getVarByName(name).x >0:
                        cycle.append(distances_data.columns[j])
                        cycle.append(" > ")
                        k=j
        cycle.append(cycle[0])
        return(cycle)
    keys_list = list(d_starting_paths.keys())
    k=1

    print('\x1b[1;31m'+'Routes Traversed:'+'\x1b[0m')
    full_distance = 0
    full_duration = 0
    bus = 1
    bus_dict = {}
    for i in keys_list:
        filter_list = [j for j in keys_list if i!=j]
        d_starting_paths[i] = cycle(filter_list)
        stops = [i for i in d_starting_paths[i] if i != ' > ']

        distance = 0
        duration = 0
        bus_dict[f"Bus_{bus}"] = stops
        bus += 1
        for j in range(len(stops)-1):
            distance += dist_array[distances_data.columns.get_loc(stops[j]), distances_data.columns.get_loc(stops[j+1])]
            duration += dur_array[duration_data.columns.get_loc(stops[j]), duration_data.columns.get_loc(stops[j+1])]
        print("\nBus ", k," (", len(stops)-2,  f" Stops, {distance/1000:.2f} km, {duration:.2f} mins):\n", *d_starting_paths[i], sep="")
        full_distance += distance
        full_duration += duration
        k+=1
    print('\x1b[1;31m'+'Key Results:'+'\x1b[0m')
    print(f"Total length of all routes: {full_distance/1000:.2f} km \nAverage duration of all routes: {full_duration/(k-1):.2f} mins \nObjective Value: {int(model.objVal):,}")
    #Bus routes to csv
    bus_route_list = []
    for i in bus_dict:
        df = pd.DataFrame(bus_dict[i])
        df.columns = [i]
        bus_route_list.append(df)
        df.to_csv(f"buses_{buses}_route_{i}.csv", index=False)


In [10]:
results(buses)

{'0>54': None}
{'0>54': None, '0>61': None}
Routes Traversed:

Bus 1 (32 Stops, 18.21 km, 57.18 mins):
65009 > 65349 > 65339 > 65409 > 65401 > 65681 > 65689 > 65661 > 65651 > 65641 > 65149 > 65579 > 65559 > 65551 > 65571 > 65439 > 65431 > 65331 > 65341 > 65229 > 65289 > 65241 > 65151 > 65089 > 65099 > 65301 > 65381 > 65281 > 65221 > 65351 > 65091 > 65081 > 65201 > 65009

Bus 2 (37 Stops, 17.89 km, 53.80 mins):
65009 > 65071 > 65209 > 65079 > 65159 > 65169 > 65179 > 65171 > 65419 > 65321 > 65181 > 65261 > 65429 > 65161 > 65249 > 65389 > 65311 > 65411 > 65271 > 65279 > 65269 > 65231 > 65521 > 65529 > 65239 > 65189 > 65569 > 65561 > 65329 > 65379 > 65371 > 65319 > 65399 > 65391 > 65309 > 65359 > 65251 > 65259 > 65009
Key Results:
Total length of all routes: 36.10 km 
Average duration of all routes: 55.49 mins 
Objective Value: 36,097


## For 3 buses and 40 mins per route

In [11]:
buses = 3
max_duration = 39.95
model = model_setup(buses, max_duration)
time_1 = datetime.now() # Start-time
model.optimize() # Run model optimization
time_2 = datetime.now() # End-time


Gurobi Optimizer version 9.5.2 build v9.5.2rc0 (mac64[arm])
Thread count: 10 physical cores, 10 logical processors, using up to 10 threads
Optimize a model with 10008 rows, 5040 columns and 43747 nonzeros
Model fingerprint: 0x6cc3b7d0
Variable types: 70 continuous, 4970 integer (4900 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+04]
  Objective range  [2e+02, 7e+03]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+04]
Presolve removed 139 rows and 72 columns
Presolve time: 0.04s
Presolved: 9869 rows, 4968 columns, 43332 nonzeros
Variable types: 69 continuous, 4899 integer (4830 binary)

Root relaxation: objective 3.515422e+04, 397 iterations, 0.01 seconds (0.03 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 35154.2154    0   50          - 35154.2154      -     -    0s
     0     0 35684.7672    0   89          - 35684.7672     

In [12]:
results(buses)

{'0>18': None}
{'0>18': None, '0>54': None}
{'0>18': None, '0>54': None, '0>61': None}
Routes Traversed:

Bus 1 (15 Stops, 12.27 km, 38.73 mins):
65009 > 65401 > 65681 > 65689 > 65661 > 65651 > 65641 > 65149 > 65579 > 65559 > 65551 > 65571 > 65439 > 65431 > 65331 > 65341 > 65009

Bus 2 (30 Stops, 12.50 km, 39.52 mins):
65009 > 65349 > 65339 > 65409 > 65259 > 65229 > 65099 > 65301 > 65381 > 65241 > 65151 > 65089 > 65289 > 65389 > 65311 > 65411 > 65271 > 65279 > 65419 > 65321 > 65181 > 65261 > 65429 > 65161 > 65249 > 65281 > 65221 > 65351 > 65091 > 65081 > 65201 > 65009

Bus 3 (24 Stops, 12.90 km, 39.93 mins):
65009 > 65071 > 65209 > 65079 > 65159 > 65169 > 65179 > 65171 > 65269 > 65231 > 65521 > 65529 > 65239 > 65189 > 65569 > 65561 > 65329 > 65379 > 65371 > 65319 > 65399 > 65391 > 65309 > 65359 > 65251 > 65009
Key Results:
Total length of all routes: 37.67 km 
Average duration of all routes: 39.39 mins 
Objective Value: 37,667


## For 4 buses and 36 mins per route

In [13]:
buses = 4
max_duration = 36
model = model_setup(buses, max_duration)
time_1 = datetime.now() # Start-time
model.optimize() # Run model optimization
time_2 = datetime.now() # End-time

Gurobi Optimizer version 9.5.2 build v9.5.2rc0 (mac64[arm])
Thread count: 10 physical cores, 10 logical processors, using up to 10 threads
Optimize a model with 10008 rows, 5040 columns and 43747 nonzeros
Model fingerprint: 0x734ece5e
Variable types: 70 continuous, 4970 integer (4900 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+04]
  Objective range  [2e+02, 7e+03]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+04]
Presolve removed 139 rows and 72 columns
Presolve time: 0.04s
Presolved: 9869 rows, 4968 columns, 43332 nonzeros
Variable types: 69 continuous, 4899 integer (4830 binary)

Root relaxation: objective 3.664717e+04, 426 iterations, 0.01 seconds (0.03 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 36647.1707    0   40          - 36647.1707      -     -    0s
     0     0 37172.1611    0  109          - 37172.1611     

In [14]:
results(buses)

{'0>51': None}
{'0>51': None, '0>54': None}
{'0>51': None, '0>54': None, '0>59': None}
{'0>51': None, '0>54': None, '0>59': None, '0>61': None}
Routes Traversed:

Bus 1 (25 Stops, 12.35 km, 35.72 mins):
65009 > 65209 > 65089 > 65289 > 65389 > 65311 > 65411 > 65269 > 65231 > 65521 > 65529 > 65239 > 65189 > 65569 > 65561 > 65329 > 65379 > 65371 > 65319 > 65399 > 65391 > 65309 > 65359 > 65229 > 65081 > 65201 > 65009

Bus 2 (10 Stops, 5.95 km, 19.78 mins):
65009 > 65349 > 65339 > 65579 > 65559 > 65551 > 65571 > 65439 > 65431 > 65331 > 65341 > 65009

Bus 3 (13 Stops, 10.48 km, 35.60 mins):
65009 > 65351 > 65091 > 65221 > 65251 > 65401 > 65681 > 65689 > 65661 > 65651 > 65641 > 65149 > 65409 > 65259 > 65009

Bus 4 (21 Stops, 10.49 km, 34.83 mins):
65009 > 65071 > 65079 > 65159 > 65169 > 65179 > 65171 > 65419 > 65321 > 65181 > 65261 > 65271 > 65279 > 65429 > 65161 > 65249 > 65281 > 65099 > 65301 > 65381 > 65241 > 65151 > 65009
Key Results:
Total length of all routes: 39.27 km 
Average duration

## For 5 buses within 30 mins per route

In [8]:
buses = 5
max_duration = 30
model = model_setup(buses, max_duration)
time_1 = datetime.now() # Start-time
model.optimize() # Run model optimization
time_2 = datetime.now() # End-time

Set parameter Username
Academic license - for non-commercial use only - expires 2023-10-16
Gurobi Optimizer version 9.5.2 build v9.5.2rc0 (mac64[arm])
Thread count: 10 physical cores, 10 logical processors, using up to 10 threads
Optimize a model with 9939 rows, 5040 columns and 43609 nonzeros
Model fingerprint: 0xc57c3a78
Variable types: 70 continuous, 4970 integer (4900 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+04]
  Objective range  [2e+02, 7e+03]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+04]
Presolve removed 139 rows and 72 columns
Presolve time: 0.03s
Presolved: 9800 rows, 4968 columns, 43194 nonzeros
Variable types: 69 continuous, 4899 integer (4830 binary)

Root relaxation: objective 3.818001e+04, 450 iterations, 0.02 seconds (0.04 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 38180.0094    0   50          - 

In [16]:
results(buses)

{'0>18': None}
{'0>18': None, '0>54': None}
{'0>18': None, '0>54': None, '0>61': None}
{'0>18': None, '0>54': None, '0>61': None, '0>62': None}
{'0>18': None, '0>54': None, '0>61': None, '0>62': None, '0>66': None}
Routes Traversed:

Bus 1 (8 Stops, 8.50 km, 28.47 mins):
65009 > 65401 > 65681 > 65689 > 65661 > 65651 > 65641 > 65149 > 65409 > 65009

Bus 2 (10 Stops, 5.95 km, 19.78 mins):
65009 > 65349 > 65339 > 65579 > 65559 > 65551 > 65571 > 65439 > 65431 > 65331 > 65341 > 65009

Bus 3 (17 Stops, 7.70 km, 28.38 mins):
65009 > 65071 > 65209 > 65089 > 65289 > 65241 > 65249 > 65389 > 65399 > 65391 > 65381 > 65281 > 65221 > 65351 > 65091 > 65229 > 65081 > 65201 > 65009

Bus 4 (16 Stops, 9.95 km, 28.38 mins):
65009 > 65079 > 65159 > 65169 > 65179 > 65171 > 65269 > 65231 > 65521 > 65529 > 65239 > 65261 > 65271 > 65279 > 65429 > 65161 > 65151 > 65009

Bus 5 (18 Stops, 8.94 km, 29.77 mins):
65009 > 65099 > 65301 > 65311 > 65411 > 65419 > 65321 > 65181 > 65189 > 65569 > 65561 > 65329 > 65379 > 

## Baseline - 1 bus and route within 106 mins

In [15]:
buses = 1
max_duration = 200
L = 70
model = model_setup(buses, max_duration)
time_1 = datetime.now() # Start-time
model.optimize() # Run model optimization
time_2 = datetime.now() # End-time

Gurobi Optimizer version 9.5.2 build v9.5.2rc0 (mac64[arm])
Thread count: 10 physical cores, 10 logical processors, using up to 10 threads
Optimize a model with 10008 rows, 5040 columns and 43747 nonzeros
Model fingerprint: 0x6a122804
Variable types: 70 continuous, 4970 integer (4900 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+04]
  Objective range  [2e+02, 7e+03]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+04]
Presolve removed 139 rows and 72 columns
Presolve time: 0.05s
Presolved: 9869 rows, 4968 columns, 43332 nonzeros
Variable types: 69 continuous, 4899 integer (4830 binary)

Root relaxation: objective 3.276996e+04, 580 iterations, 0.03 seconds (0.07 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 32769.9571    0   51          - 32769.9571      -     -    0s
     0     0 33259.3456    0   87          - 33259.3456     

In [16]:
results(buses)

{'0>54': None}
Routes Traversed:

Bus 1 (69 Stops, 35.22 km, 105.75 mins):
65009 > 65349 > 65339 > 65409 > 65259 > 65351 > 65091 > 65221 > 65251 > 65401 > 65681 > 65689 > 65661 > 65651 > 65641 > 65149 > 65579 > 65559 > 65551 > 65571 > 65439 > 65431 > 65331 > 65341 > 65071 > 65209 > 65079 > 65159 > 65169 > 65179 > 65171 > 65419 > 65321 > 65181 > 65261 > 65429 > 65161 > 65249 > 65281 > 65099 > 65301 > 65381 > 65241 > 65151 > 65089 > 65289 > 65389 > 65311 > 65411 > 65271 > 65279 > 65269 > 65231 > 65521 > 65529 > 65239 > 65189 > 65569 > 65561 > 65329 > 65379 > 65371 > 65319 > 65399 > 65391 > 65309 > 65359 > 65229 > 65081 > 65201 > 65009
Key Results:
Total length of all routes: 35.22 km 
Average duration of all routes: 105.75 mins 
Objective Value: 35,219


# Exporting Data

In [21]:
##export csv routes for plotting in google maps

pung_df = pd.read_csv('punggol_busstops.csv')

for no_of_buses in range(0,5):
    for bus in range(no_of_buses+1):
        
        bus_route_df = pd.read_csv(f'buses_{no_of_buses+1}_route_Bus_{bus+1}.csv')
        bus_route_df = pd.merge(bus_route_df, pung_df, left_on=f'Bus_{bus+1}', right_on='stop_code', how='left')
        bus_route_df = bus_route_df[['Name']].rename(columns={'Name':f'Name_route{bus+1}'})
        bus_route_df.to_csv(f"buses_{no_of_buses+1}_name_route_{bus+1}.csv", index=False)
    
